In [1]:
import random

from distributed_processing.serializers import DummySerializer
from distributed_processing.client import Client
from distributed_processing.filesystem_connector import FileSystemConnector


In [2]:
CURRO = True

if CURRO:
    NS_PATH ="G:\\fs_namespaces\\prueba_distribuida"
    NS_PATH ="C:\\fs_namespaces\\prueba_distribuida"

else:
    NS_PATH = "/home/augusto/python/notebooks/fs_namespaces/prueba_distribuida"


In [3]:
import logging
logging.getLogger("distributed_processing").setLevel(logging.DEBUG)
#logging.getLogger("distributed_processing.filesystem_connector").setLevel(logging.DEBUG)

In [4]:
fs_connector = FileSystemConnector(NS_PATH)
fs_connector.with_watchdog=True
fs_connector.pop_watchdog_timeout = 10

client = Client(DummySerializer(), fs_connector, check_registry="cache")

registry_lock set


registry_lock unset
nclients_lock set
nclients_lock unset
Client with id: fs_client_9
Results queue: fs_client_9_responses


In [5]:
fs_connector.methods_registry()

registry_lock set
registry_lock unset


{'add': ['requests_cola_1'],
 'div': ['requests_cola_1'],
 'eval_py_function': ['requests_py_eval'],
 'mul': ['requests_cola_1']}

In [6]:
y = client.rpc_async("add", [1, 0])

Sent request with id: fs_client_9:1 to queue requests_cola_1


In [7]:
y.get()

1

In [8]:
client.check_registry ="always"

try:
    y = client.rpc_async("fake", [1, 0])
except Exception as e:
    print(e)

'method_queues_fake'


In [9]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

try:
    y.get()
except Exception as e:
    print(e)


Sent request with id: fs_client_9:2 to queue requests_cola_1


Error -32601 : The method does not exist/is not available.

 


In [10]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

y.safe_get(default="Esto es un error")

Sent request with id: fs_client_9:3 to queue requests_cola_1


'Esto es un error'

In [11]:
client.check_registry ="cache"

In [12]:
x = client.rpc_async("div", [1, 0])

Sent request with id: fs_client_9:4 to queue requests_cola_1


In [13]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32603 : Internal RPC error.

 Traceback (most recent call last):
  File "G:\python\distributed_processing\python\distributed_processing\worker.py", line 149, in _process_single_request
    result = func[request["method"]](*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\agarcia\AppData\Local\Temp\ipykernel_20736\1866336200.py", line 8, in div
    return x / y
           ~~^~~
ZeroDivisionError: division by zero



In [14]:
client.rpc_sync("add", [1, 1])

Sent request with id: fs_client_9:5 to queue requests_cola_1


2

In [15]:
def f(x,y): return x + y

y = client.rpc_async_fn(f, [1, 2.0])

Sent request with id: fs_client_9:6 to queue requests_py_eval


In [16]:
y.get()

3.0

In [17]:
client.rpc_sync_fn(f, [3.0, 2.0])

Sent request with id: fs_client_9:7 to queue requests_py_eval


5.0

In [41]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)
    fs.append(client.rpc_async(t[0], t[1]))

Sent request with id: fs_client_9:61 to queue requests_cola_1
Sent request with id: fs_client_9:62 to queue requests_cola_1


Sent request with id: fs_client_9:63 to queue requests_cola_1
Sent request with id: fs_client_9:64 to queue requests_cola_1
Sent request with id: fs_client_9:65 to queue requests_cola_1
Sent request with id: fs_client_9:66 to queue requests_cola_1
Sent request with id: fs_client_9:67 to queue requests_cola_1
Sent request with id: fs_client_9:68 to queue requests_cola_1


('div', [0.21861316807493014, 0.6555079784846946], {})
('add', [0.5378880682634101, 0.8144833232797576], {})
('mul', [0.6696128016002426, 0.7225986240249452], {})
('add', [0.5609003531352541, 0.48596425868353843], {})
('div', [0.855759719144058, 0.7556443896992038], {})
('div', [0.11307516930554684, 0.9382068372231882], {})
('add', [0.5310446207870345, 0.4965417023328643], {})
('div', [0.12747335396747672, 0.6398222538622869], {})
('div', [0.02612810103003549, 0.7728322458610023], {})


Sent request with id: fs_client_9:69 to queue requests_cola_1
Sent request with id: fs_client_9:70 to queue requests_cola_1


('add', [0.9401441799985025, 0.5090111055476992], {})


In [42]:
[f.get() for f in fs]

[0.3335019179786147,
 1.3523713915431677,
 0.483861289065824,
 1.0468646118187925,
 1.1324900056291118,
 0.12052264470829858,
 1.0275863231198987,
 0.1992324480712946,
 0.0338082438588293,
 1.4491552855462018]

In [43]:
fs = client.rpc_multi_async(tp)

Sent request with id: fs_client_9:71 to queue requests_cola_1
Sent request with id: fs_client_9:72 to queue requests_cola_1


Sent request with id: fs_client_9:73 to queue requests_cola_1
Sent request with id: fs_client_9:74 to queue requests_cola_1
Sent request with id: fs_client_9:75 to queue requests_cola_1
Sent request with id: fs_client_9:76 to queue requests_cola_1
Sent request with id: fs_client_9:77 to queue requests_cola_1
Sent request with id: fs_client_9:78 to queue requests_cola_1
Sent request with id: fs_client_9:79 to queue requests_cola_1
Sent request with id: fs_client_9:80 to queue requests_cola_1


In [44]:
[f.status for f in fs]

['PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING']

In [22]:
client._fetch_all_available_responses()

In [23]:
client.pending

{'fs_client_9:18': 1745488997.8393955,
 'fs_client_9:19': 1745488997.8548253,
 'fs_client_9:20': 1745488997.8716102,
 'fs_client_9:21': 1745488997.900639,
 'fs_client_9:22': 1745488997.9218779,
 'fs_client_9:23': 1745488997.940089,
 'fs_client_9:24': 1745488997.959467,
 'fs_client_9:25': 1745488997.989041,
 'fs_client_9:26': 1745488998.0059712,
 'fs_client_9:27': 1745488998.0260777}

In [24]:
[f.get() for f in fs]

[1.6053825103201924,
 0.21251333476294876,
 2.501937321749925,
 1.474165687291201,
 1.2075749299779535,
 3.469194038690695,
 0.8636156312848585,
 1.9391721090064737,
 1.7402308591958786,
 1.8327709439917517]

In [25]:
fs = client.rpc_batch_async(tp)

Sent batch request with 10 requests to requests_cola_1


In [26]:
[f.get() for f in fs]

[1.6053825103201924,
 0.21251333476294876,
 2.501937321749925,
 1.474165687291201,
 1.2075749299779535,
 3.469194038690695,
 0.8636156312848585,
 1.9391721090064737,
 1.7402308591958786,
 1.8327709439917517]

In [27]:
client.rpc_batch_sync(tp)

Sent batch request with 10 requests to requests_cola_1


[1.6053825103201924,
 0.21251333476294876,
 2.501937321749925,
 1.474165687291201,
 1.2075749299779535,
 3.469194038690695,
 0.8636156312848585,
 1.9391721090064737,
 1.7402308591958786,
 1.8327709439917517]

In [28]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "fake"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)

('fake', [0.09430896565055147, 0.9647305543462525], {})
('fake', [0.28800946042304476, 0.8276562419606465], {})
('mul', [0.9423354137874825, 0.6996229583025027], {})
('fake', [0.9922002436153445, 0.9393722237018601], {})
('mul', [0.6037835043301335, 0.8048813898097604], {})
('add', [0.00021623767102418867, 0.16282075524186568], {})
('mul', [0.7536937689005387, 0.33968283498773233], {})
('add', [0.12832001014866667, 0.20934946148623435], {})
('add', [0.8396723477438003, 0.5872105314515605], {})
('div', [0.6538224424872068, 0.39422507532179774], {})


In [29]:
tp

[('fake', [0.09430896565055147, 0.9647305543462525], {}),
 ('fake', [0.28800946042304476, 0.8276562419606465], {}),
 ('mul', [0.9423354137874825, 0.6996229583025027], {}),
 ('fake', [0.9922002436153445, 0.9393722237018601], {}),
 ('mul', [0.6037835043301335, 0.8048813898097604], {}),
 ('add', [0.00021623767102418867, 0.16282075524186568], {}),
 ('mul', [0.7536937689005387, 0.33968283498773233], {}),
 ('add', [0.12832001014866667, 0.20934946148623435], {}),
 ('add', [0.8396723477438003, 0.5872105314515605], {}),
 ('div', [0.6538224424872068, 0.39422507532179774], {})]

In [30]:
client.check_registry ="never"
client.set_default_queue("cola_1")

fs = client.rpc_batch_async(tp)

Sent batch request with 10 requests to requests_cola_1


In [31]:
[f.safe_get() for f in fs]

[None,
 None,
 0.6592794899072115,
 None,
 0.48597410610944536,
 0.16303699291288987,
 0.25601683613272375,
 0.337669471634901,
 1.426882879195361,
 1.6585003933438407]

In [32]:
client.responses

{}

In [33]:
client.check_registry ="cache"

In [34]:
try:
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [35]:
try:    
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [36]:
client.check_registry="never"
client.set_default_queue("cola_1")

x = client.rpc_async("kk", [1, 0])

Sent request with id: fs_client_9:58 to queue requests_cola_1


In [37]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 


In [38]:
y = client.rpc_async("add", [1, 0])

Sent request with id: fs_client_9:59 to queue requests_cola_1


In [39]:
y.get(5)

1

In [40]:
def f(x,y): return x + y

client.check_registry="never"
y = client.rpc_async_fn(f, [1, 2.0])
try:
    y.get()
except Exception as e:
    print(e)

Sent request with id: fs_client_9:60 to queue requests_cola_1


Error -32601 : The method does not exist/is not available.

 
